# Objectives


* To explore the convolutional and long short-term neural networks from Week 8 lecture and apply them for forecast of number of days of ground frost and snow based on other weather variables.
> **Remember**: It is your responsibility as a machine learning scientist to read documentations for each library function in the code to thoroughly understand what it is doing, how it serves the purpose highlighted in the code comments, and other parameters that could be set.


# Section 1 - Load new UK Met (60km, 2010-2022) data

Although the same UK Met dataset will be used, the format of the dataset for this week is different from that in Week 3.

1. You need to first download the data before you can get started. Download from the Week 8 page for the module, on Canvas (see 'Week 8 Lab Dataset' on the page). The file you download will be named *curated_data_24month_2010-2022_nonans.csv*.

2. Explore the dataset (see Section 1 of the Week 3 lab notebook for where to get started).

3. Then, use the file menu in Google Colab to upload the file to your Colab directory. Once upload is complete, you should be able to see the file on the listed contents of your Colab directory.

4. You can now run the code in the cell below to load the data.

In [ ]:
import csv
import numpy


!ls  /content

data_file_full_path = "/content/curated_data_24month_2010-2022_nonans.csv"

data_as_list = []

# load the dataset
with open(data_file_full_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')

    row_count = 0
    for row in csv_reader:

      if row_count > 0:
        data_as_list.append([float(val) for val in row])
      row_count += 1
data = numpy.array(data_as_list)

# check its shape
print("\n The dataset has shape: "+str(data.shape))


# get features and labels from the data
# based on the objectives (see the Objectives section)
ground_frost_24_col = 27
hurs_1_to_23_col = numpy.arange(ground_frost_24_col+1, ground_frost_24_col+24)
psl_1_to_23_col = hurs_1_to_23_col + 23
sun_1_to_23_col = psl_1_to_23_col + 23
pv_1_to_23_col = sun_1_to_23_col + 23
wind_1_to_23_col = pv_1_to_23_col + 23
tas_1_to_23_col = wind_1_to_23_col + 23
snow_24_col = 97
rain_1_to_23_col = numpy.arange(snow_24_col+1, snow_24_col+24)

# arrange to shape N x T X D
# where N = number of data instances
# T = number of timesteps
# D = number of variables
feats = numpy.stack((data[:, hurs_1_to_23_col], data[:, psl_1_to_23_col],
                     data[:, sun_1_to_23_col], data[:, pv_1_to_23_col],
                     data[:, wind_1_to_23_col], data[:, tas_1_to_23_col],
                     data[:, rain_1_to_23_col]))
feats = numpy.transpose(feats, (1, 2, 0))
ground_frost_label = data[:, ground_frost_24_col]
snow_label = data[:, snow_24_col]

# check its shape
print("\n The reshaped data has shape: "+str(feats.shape))

# take a peek
print("\n A peek at the dataset features: \n"+str(feats))
print("\n A peek at the ground frost labels: \n"+str(ground_frost_label))
print("\n A peek at the snow labels: \n"+str(snow_label))


# Section 2 - Split into training, validation, and test sets

In [ ]:
from sklearn.model_selection import train_test_split

all_ids = numpy.arange(0, feats.shape[0])

random_seed = 1

# First randomly split the data into 70:30 to get the training set
train_set_ids, rem_set_ids = train_test_split(all_ids, test_size=0.3, train_size=0.7,
                                 random_state=random_seed, shuffle=True)


# Then further split the remaining data 50:50 into validation and test sets
val_set_ids, test_set_ids = train_test_split(rem_set_ids, test_size=0.5, train_size=0.5,
                                 random_state=random_seed, shuffle=True)


train_ground_frost_labels = ground_frost_label[train_set_ids]
train_snow_labels = snow_label[train_set_ids]

val_ground_frost_labels = ground_frost_label[val_set_ids]
val_snow_labels = snow_label[val_set_ids]

test_ground_frost_labels = ground_frost_label[test_set_ids]
test_snow_labels = snow_label[test_set_ids]

# Section 3 - Scale (i.e. normalize) the input data

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# note that unlike previous weeks
# a normalization is to 0-1 range here
scaled_feats = numpy.reshape(feats, (-1, feats.shape[2]))
scaler = MinMaxScaler()
scaler.fit(scaled_feats)
scaled_feats = scaler.transform(scaled_feats)
scaled_feats = numpy.reshape(scaled_feats, (feats.shape[0], feats.shape[1], feats.shape[2]))
print("\n A peek at the scaled dataset features: \n"+str(scaled_feats))

scaled_train_data = scaled_feats[train_set_ids, :]
scaled_val_data = scaled_feats[val_set_ids, :]
scaled_test_data = scaled_feats[test_set_ids, :]

# Section 4 - Train a convolutional neural network (CNN)

In [ ]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.metrics import mean_absolute_error, mean_squared_error
from copy import deepcopy
import sys
import matplotlib.pyplot as plt



# Creating the network structure
# for a 3-layer CNN
class three_layer_CNN(nn.Module):
    def __init__(self,
                 hidden_layer_channel_sizes,
                 output_size, input_channel_size):
        super().__init__()

        self.hidden_c1 = nn.Conv1d(input_channel_size, hidden_layer_channel_sizes[0], 6, stride=1)
        self.hidden_m1 = (nn.MaxPool1d(3, stride=1))
        self.hidden_c2 = nn.Conv1d(hidden_layer_channel_sizes[0], hidden_layer_channel_sizes[1], 4, stride=1)
        self.hidden_m2 = nn.MaxPool1d(3, stride=1)
        self.hidden_c3 = nn.Conv1d(hidden_layer_channel_sizes[1], hidden_layer_channel_sizes[2], 3, stride=1)
        self.hidden_m3 = nn.MaxPool1d(3, stride=1)
        self.output = nn.Linear(output_size[0], output_size[1])
        self.relu = nn.ReLU()


    def forward(self, inputs):
        out = self.hidden_c1(torch.permute(inputs, (0, 2, 1)))
        out = self.hidden_m1(self.relu(out))
        out = self.hidden_c2(out)
        out = self.hidden_m2(self.relu(out))
        out = self.hidden_c3(out)
        out = self.hidden_m3(self.relu(out))
        out = torch.reshape(out, (out.size(0), -1))
        out = self.output(self.relu(out))
        return out



# a method for training a model
def train_model(model,
                train_set, val_set,
                epochs, learn_rate):

  # Setting up the SGD optimizer for updating the model weights
  optimizer = optim.SGD(model.parameters(), lr=learn_rate)


  # Computing cross entropy loss against the training labels
  loss_function = nn.MSELoss()



  best_model_mae = sys.float_info.max
  losses = []

  # Iterating over the dataset at two different staages:
  # 1. Iterating over the batches in the dataset (inner for loop below)
  # One complete set of iteration through the dataset (i.e. having gone over
  # all batches in the dataset at least once) = One epoch
  # 2. Iterating over the specified numeber of epochs (outer for loop below)
  for epoch in range(0, epochs):

      # Setting the model to training mode
      model.train()

      if epoch == 0:  best_model = deepcopy(model)

      for batch, (X_train, y_train) in enumerate(train_set):

        # Zeroing out the `.grad` buffers,
        # otherwise on the backward pass we'll add the
        # new gradients to the old ones.
        optimizer.zero_grad()

        # Computing the forward pass and then the loss
        train_pred = model.forward(X_train)
        train_loss = loss_function(train_pred, y_train)
        train_pred_numpy = train_pred.detach().numpy()
        train_mae = mean_absolute_error(y_train[:], train_pred_numpy[:])

        # Computing the model parameters' gradients
        # and propagating the loss backwards through the network.
        train_loss.backward()

        # Updating the model parameters using those gradients
        optimizer.step()

      # Evaluating on the validation set
      model.eval()
      for batch, (X_val, y_val) in enumerate(val_set):
        val_pred = model.forward(X_val)
        val_loss = loss_function(val_pred, y_val)
        val_pred_numpy = val_pred.detach().numpy()
        val_mae = mean_absolute_error(y_val[:], val_pred_numpy[:])

      if val_mae <= best_model_mae:
        best_model_mae = val_mae
        best_model = deepcopy(model)
        print('Found improvement in performance. New model saved.')

      # How well the network does at the end of an epoch
      # is an indication of how well training is progressing
      print("epoch: {} - train loss: {:.4f} train mae: {:.2f} val loss: {:.4f} val mae: {:.2f}".format(
          epoch,
          train_loss.item(),
          train_mae,
          val_loss.item(),
          val_mae))

      losses.append([train_loss.item(), val_loss.item()])

  return best_model, losses


# a method for evaluating a model
# including the metrics earlier prepared
# and a graph of the above loss per epoch
def evaluate_model(model,
                test_set,
                training_losses, plot=True, text=''):

  model.eval()
  for batch, (X_test, y_test) in enumerate(test_set):
    test_pred = model.forward(X_test)
    test_pred_numpy = test_pred.detach().numpy()
    test_mae = mean_absolute_error(y_test[:], test_pred_numpy[:])
    print("\n Mean absolute error on the test set{:s}: {:2.2f}".format(text, test_mae))
    print()

  if plot:
    fig, ax = plt.subplots()
    training_losses = numpy.array(training_losses)
    ax.plot(training_losses[:, 0], 'b-', label='training loss')
    ax.plot(training_losses[:, 1], 'k-', label='validation loss')
    plt.legend(loc='upper right')
    plt.xlabel("epochs")
    plt.show




# A class for managing the data for training the model
class MetDataset(Dataset):
    def __init__(self, feats, labels):
        # Convert features from numpy arrays to PyTorch tensors
        self.feats = torch.tensor(feats, dtype=torch.float32)


        # Convert labels from numpy arrays to PyTorch tensors
        self.labels = torch.tensor(numpy.reshape(labels, (-1, 1)), dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):

        return self.feats[idx, :], self.labels[idx, :]


In [ ]:
import random

# To ensure reproducibility
# for PyTorch operations that use random numbers internally
random.seed(random_seed)
torch.manual_seed(random_seed)
numpy.random.seed(random_seed)

# Create an instance of the CNN network
input_channel_size_cnn = 7
hidden_layer_sizes_cnn = [7, 30, 7]
output_sizes_cnn = [hidden_layer_sizes_cnn[2]*7, 1]
model_CNN = three_layer_CNN(hidden_layer_sizes_cnn, output_sizes_cnn, input_channel_size_cnn)


# Set hyperparameters
num_epochs_CNN = 100
learning_rate_CNN = 0.0005
batch_size = 50


# Set up the data loading by batches
# With the test and validation sets having only one batch
train_set = MetDataset(scaled_train_data, train_ground_frost_labels)
train_dataloader = DataLoader(train_set, batch_size=batch_size)

val_set = MetDataset(scaled_val_data, val_ground_frost_labels)
val_dataloader = DataLoader(val_set, batch_size=len(val_set))

test_set = MetDataset(scaled_test_data, test_ground_frost_labels)
test_dataloader = DataLoader(test_set, batch_size=len(test_set))



# train the model
model_CNN, train_val_losses_CNN = train_model(model_CNN, train_dataloader,
                                                          val_dataloader,
                                                          num_epochs_CNN,
                                                          learning_rate_CNN)

# evaluate the model
evaluate_model(model_CNN, test_dataloader, train_val_losses_CNN)



# Section 5 - Explore the CNN

* For each layer of the CNN in Section 4, calculate the dimension of the output of the layer. (Hint: See Week 8 lecture slides for the formulae for computing the output dimension of a convolutional layer. The same formula applies to pooling layers.)

* Based on your calculations above and the structure of the CNN, how many parameters does the model have? How does this compare with the number of parameters if each convolutional layer was instead a fully connected layer.

* Perform data augmentation (see Week 5 lecture), e.g.
  * dropout along the time dimension,
  * dropout along the variable dimension,
  * random noise along the time dimension.

* How does the augmentation affect generalisation of the model?

# Section 6 - Train a long short-term memory neural network (LSTMNN)

* Create and train a three-layer LSTMNN model (see https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html for the documentation for the PyTorch LSTM layer)

* How many weights does the LSTMNN have?

* How does your model perform in comparison with the CNN in Section 4?